In [1]:
from keras.utils import to_categorical
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters
import networkx as nx

In [2]:
from sklearn.datasets import load_svmlight_file


def get_data(data):
    data = load_svmlight_file(data)
    return data[0].toarray(), data[1]
data = "./abalone_scale.txt"
X, y = get_data(data)
#y[y == -1] = 0
# Convert labels to one-hot encoding
#y = to_categorical(y)

#data = "./a1a_t"
#X_test,y_test = get_data(data)
#y_test[y_test == -1] = 0
#y_test = to_categorical(y_test)


In [3]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.shape[0])
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split

In [4]:
no_users = 30

In [5]:
X, y = random_split(X, y, 3, 42)

In [6]:
X1 = X[0][:, 0:4]
X2 = X[1][:, 2: 6]
X3 = X[2][:, 3:]
y1 = y[0]
y2 = y[1]
y3 = y[2]

X_test = [np.concatenate([X[1], X[2]], axis=0), np.concatenate([X[0], X[2]], axis=0), np.concatenate([X[0], X[1]], axis=0)]
y_test = [np.concatenate((y[1], y[2])), np.concatenate((y[0], y[2])), np.concatenate((y[0], y[1]))]


subset_ranges = [np.arange(0, 4), np.arange(2, 6), np.arange(3, 8)]
subset_lengths = [subset_ranges[0].shape[0], subset_ranges[1].shape[0], subset_ranges[2].shape[0]]

In [7]:
len(X1)

1393

In [8]:
X_test[0].shape

(2784, 8)

In [9]:
X_test[0].shape

(2784, 8)

In [10]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout, pout],[pout, pin, pout],[pout, pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs, seed=0)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10, 10]
#features_sizes = [8, 7, 6, 5]
pin = 0.5
pout = 0.1
seed = 0
alpha = 1e-2
lamda = 0#1e-1#1e-3#1e-1#1e-3
eta = 1 * 1e-2
d0 = min(subset_lengths)
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

# Set a random seed for reproducibility
seed = 17
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [11]:
d0

8

In [12]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [13]:
y1.shape

(1393,)

In [14]:
datapoints = {}
count = 0

X1, y1 = random_split(X1, y1, 10, 42)
X2, y2 = random_split(X2, y2, 10, 42)
X3, y3 = random_split(X3, y3, 10, 42)

X_train = [X1, X2, X3]
y_train = [y1, y2, y3]
input_sizes = [X1[0].shape[1], X2[0].shape[1], X3[0].shape[1]]

for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        
        test_features = X_train[i][j]#X_test[:, subset_ranges[i]]
        test_label = y_train[i][j]#y_test
        datapoints[count] = {
                'features': X_train[i][j],
                'degree': node_degree(count, G),
                'label': y_train[i][j],
                'neighbors': get_neighbors(count, G),
                'input_size': X_train[i][j].shape[1],
                'test_features':X_test[i][:, subset_ranges[i]],
                'test_labels': y_test[i]
            }
        count += 1

In [15]:
datapoints[21]['test_features'].shape

(2785, 8)

In [16]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [17]:

class MLP_Net(nn.Module):
    def __init__(self, input_size, num_classes, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(input_size, num_classes, bias=True)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        output = self.fc1(x)#F.softmax(self.fc1(x), dim=1)  # Applying softmax along the second dimension
        return output

In [18]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [19]:
model = MLP_Net(datapoints[21]["input_size"], 1, user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[21]["features"], datapoints[21]["label"]), batch_size=50, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(1000):
    for j, (x, y) in zip(range(1), dataloader):
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)


        
        # Calculate accuracy
        #_, predicted = torch.max(yhat, 1)
        #_, predicted_true = torch.max(y, 1)
        #correct_predictions = (predicted == predicted_true).sum().item()
         

        loss = criterion(yhat.squeeze(), y)
        
        
        loss.backward()
        print(i, loss.detach())
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

0 tensor(93.4025)
1 tensor(121.6946)
2 tensor(102.6583)
3 tensor(82.3093)
4 tensor(73.0132)
5 tensor(92.3217)
6 tensor(80.1299)
7 tensor(57.3422)
8 tensor(65.2588)
9 tensor(58.0577)
10 tensor(40.5504)
11 tensor(51.3492)
12 tensor(50.2891)
13 tensor(52.1893)
14 tensor(55.1758)
15 tensor(27.6117)
16 tensor(31.0062)
17 tensor(35.5602)
18 tensor(32.8030)
19 tensor(24.6145)
20 tensor(25.1938)
21 tensor(24.8841)
22 tensor(28.8012)
23 tensor(25.2397)
24 tensor(28.0119)
25 tensor(22.4082)
26 tensor(18.4860)
27 tensor(21.7617)
28 tensor(26.3294)
29 tensor(17.7479)
30 tensor(21.5105)
31 tensor(30.6467)
32 tensor(17.9162)
33 tensor(15.8744)
34 tensor(20.9609)
35 tensor(18.4591)
36 tensor(16.5488)
37 tensor(13.0537)
38 tensor(16.8153)
39 tensor(15.0364)
40 tensor(19.5861)
41 tensor(10.0754)
42 tensor(11.2041)
43 tensor(10.7943)
44 tensor(16.9486)
45 tensor(22.1211)
46 tensor(10.7859)
47 tensor(14.2005)
48 tensor(20.1441)
49 tensor(10.5829)
50 tensor(22.5816)
51 tensor(10.2437)
52 tensor(16.6312)
5

439 tensor(9.4729)
440 tensor(5.3435)
441 tensor(4.2114)
442 tensor(10.4373)
443 tensor(8.4458)
444 tensor(10.3180)
445 tensor(7.3400)
446 tensor(5.0345)
447 tensor(10.2591)
448 tensor(14.1265)
449 tensor(7.1168)
450 tensor(10.3710)
451 tensor(9.2006)
452 tensor(5.4707)
453 tensor(11.8797)
454 tensor(9.3633)
455 tensor(5.8007)
456 tensor(4.8002)
457 tensor(10.9880)
458 tensor(5.7708)
459 tensor(11.2766)
460 tensor(7.2226)
461 tensor(12.4360)
462 tensor(10.8187)
463 tensor(6.8543)
464 tensor(7.0458)
465 tensor(6.4301)
466 tensor(6.3984)
467 tensor(6.2020)
468 tensor(5.4771)
469 tensor(6.8973)
470 tensor(11.4558)
471 tensor(8.1193)
472 tensor(14.7236)
473 tensor(6.6412)
474 tensor(10.7313)
475 tensor(9.8787)
476 tensor(4.7234)
477 tensor(7.1869)
478 tensor(10.2079)
479 tensor(7.2461)
480 tensor(7.9890)
481 tensor(6.7481)
482 tensor(12.0549)
483 tensor(9.2832)
484 tensor(9.9671)
485 tensor(6.9775)
486 tensor(6.5490)
487 tensor(11.9139)
488 tensor(8.0578)
489 tensor(7.1097)
490 tensor(11.8

900 tensor(7.2022)
901 tensor(6.9912)
902 tensor(10.9864)
903 tensor(4.6179)
904 tensor(9.5163)
905 tensor(6.4326)
906 tensor(7.1401)
907 tensor(7.8353)
908 tensor(7.3568)
909 tensor(4.9576)
910 tensor(12.2050)
911 tensor(8.0864)
912 tensor(5.3518)
913 tensor(9.5107)
914 tensor(7.0288)
915 tensor(4.7146)
916 tensor(6.3132)
917 tensor(7.7449)
918 tensor(7.4608)
919 tensor(7.0597)
920 tensor(4.7534)
921 tensor(6.5064)
922 tensor(5.9731)
923 tensor(6.2760)
924 tensor(11.1439)
925 tensor(10.4024)
926 tensor(5.6694)
927 tensor(8.4276)
928 tensor(7.7747)
929 tensor(10.2556)
930 tensor(7.3119)
931 tensor(7.8629)
932 tensor(6.1120)
933 tensor(7.3113)
934 tensor(6.3813)
935 tensor(8.3312)
936 tensor(6.6076)
937 tensor(8.5386)
938 tensor(8.9513)
939 tensor(7.9242)
940 tensor(5.4387)
941 tensor(10.2702)
942 tensor(13.1192)
943 tensor(6.1367)
944 tensor(10.7050)
945 tensor(7.1904)
946 tensor(9.4496)
947 tensor(6.0622)
948 tensor(13.3876)
949 tensor(7.8252)
950 tensor(9.2321)
951 tensor(8.2854)
952

In [20]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=alpha)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            #model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output.squeeze(), labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[model.user_id][j] - projected_weights[j][model.user_id]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [21]:
# Preparing projection matrices
models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=1, user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, models, first_run=True):
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        # Specify the dimensions of the rectangular matrix
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]

                        # Generate random values for the diagonal from a normal distribution
                        diag_values = torch.ones(min(row, column))

                        # Create a rectangular matrix with diagonal elements
                        mat = torch.diag(diag_values)

                        # If the matrix is larger than the diagonal vector, fill the remaining elements with zeros
                       
                        mat = torch.cat((mat, torch.zeros(row, column - row)), dim=1)

                        

                        # Append the matrix to the list
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights, models)



In [22]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["test_features"], dataset["test_labels"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output.squeeze(), labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #_, predicted_true = torch.max(labels, 1)
        #correct += pred.eq(predicted_true.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    #correct /= len(test_loader.dataset)
    
    return test_loss#, correct

In [23]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=1, user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=1, user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

in_cluster_proj_norm = []
out_cluster_proj_norm = []
in_cluster_proj_diff_norm = []
out_cluster_proj_diff_norm = []
task_loss = {'0':[],
                '1':[],
                '2':[]}
task_rel_error = {'0':[],
            '1':[],
            '2':[]}

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    projected_weights = []
    update_ProjWeight(projection_list, projected_weights, models, first_run=False)
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            temp_mat = torch.outer(projected_weights[i][j] - projected_weights[j][i], weights)
            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
    
    in_cluster_proj_norm_round = 0
    out_cluster_proj_norm_round = 0
    in_cluster_proj_diff_round = 0
    out_cluster_proj_diff_round = 0
    in_edges = 0
    out_edges = 0
    
    for i in range(no_users//3):
        for j in G.neighbors(i):
            if j < no_users//3:
                in_edges += 1
                in_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                in_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
            else:
                out_edges += 1
                out_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                out_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
    in_cluster_proj_norm.append(in_cluster_proj_norm_round / in_edges)
    out_cluster_proj_norm.append(out_cluster_proj_norm_round / out_edges)
    in_cluster_proj_diff_norm.append(in_cluster_proj_diff_round / in_edges)
    out_cluster_proj_diff_norm.append(out_cluster_proj_diff_round / out_edges)
            
                                         
                                              
    
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    per_task_loss = []
    per_task_rel_error = []
    
    for k in range(no_users):
      
        g_loss = testing(models[k], datapoints[k], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += acc#rel_error(models[i])
        if (k + 1) % 10 == 0:
            task_loss[str(k // 10)].append(sum(per_task_loss) / 10)
            #task_rel_error[str(k // 10)].append(sum(per_task_rel_error) / 10)
            per_task_loss = []
            #per_task_rel_error = []
        per_task_loss.append(g_loss)
        #per_task_rel_error.append(acc)#rel_error(models[i]))
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f"% (test_loss[-1]))

  0%|          | 1/500 [00:01<08:31,  1.03s/it]

Training_loss 96.52692


  0%|          | 2/500 [00:01<08:11,  1.01it/s]

Training_loss 88.24042


  1%|          | 3/500 [00:02<08:11,  1.01it/s]

...

Training_loss 6.98569


100%|█████████▉| 499/500 [10:31<00:03,  3.02s/it]

Training_loss 6.98373


100%|██████████| 500/500 [10:34<00:00,  1.27s/it]

Training_loss 6.98397


In [24]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

in_cluster_proj_norm = np.array(in_cluster_proj_norm)
out_cluster_proj_norm = np.array(out_cluster_proj_norm)
in_cluster_proj_diff_norm = np.array(in_cluster_proj_diff_norm)
out_cluster_proj_diff_norm = np.array(out_cluster_proj_diff_norm)


In [25]:
'''
  0%|          | 1/2000 [00:12<6:59:49, 12.60s/it]
Training_loss 0.69317,   Accuracy 0.52177
  0%|          | 2/2000 [00:25<6:58:41, 12.57s/it]
Training_loss 0.69256,   Accuracy 0.52523
  0%|          | 3/2000 [00:37<6:55:30, 12.48s/it]
Training_loss 0.69282,   Accuracy 0.52400
  0%|          | 4/2000 [00:50<6:56:56, 12.53s/it]
Training_loss 0.69216,   Accuracy 0.52552
  0%|          | 5/2000 [01:05<7:35:34, 13.70s/it]
Training_loss 0.69178,   Accuracy 0.52765
  0%|          | 6/2000 [01:18<7:24:07, 13.36s/it]
Training_loss 0.69037,   Accuracy 0.55240
  0%|          | 7/2000 [01:32<7:32:02, 13.61s/it]
Training_loss 0.68986,   Accuracy 0.55653
  0%|          | 8/2000 [01:47<7:42:39, 13.94s/it]
Training_loss 0.68921,   Accuracy 0.56338
'''

'\n  0%|          | 1/2000 [00:12<6:59:49, 12.60s/it]\nTraining_loss 0.69317,   Accuracy 0.52177\n  0%|          | 2/2000 [00:25<6:58:41, 12.57s/it]\nTraining_loss 0.69256,   Accuracy 0.52523\n  0%|          | 3/2000 [00:37<6:55:30, 12.48s/it]\nTraining_loss 0.69282,   Accuracy 0.52400\n  0%|          | 4/2000 [00:50<6:56:56, 12.53s/it]\nTraining_loss 0.69216,   Accuracy 0.52552\n  0%|          | 5/2000 [01:05<7:35:34, 13.70s/it]\nTraining_loss 0.69178,   Accuracy 0.52765\n  0%|          | 6/2000 [01:18<7:24:07, 13.36s/it]\nTraining_loss 0.69037,   Accuracy 0.55240\n  0%|          | 7/2000 [01:32<7:32:02, 13.61s/it]\nTraining_loss 0.68986,   Accuracy 0.55653\n  0%|          | 8/2000 [01:47<7:42:39, 13.94s/it]\nTraining_loss 0.68921,   Accuracy 0.56338\n'

In [26]:
np.save( 'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)
np.save( 'in_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_norm)
np.save('out_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_norm)
np.save( 'in_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_diff_norm)
np.save('out_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_diff_norm)

In [27]:
'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'training_loss_sheave_fml0_pout0_1'

In [28]:
# task_loss dictionary
for key, value in task_loss.items():
    # Convert the list to a NumPy array
    array_loss = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('training_loss_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_loss)

# task_rel_error dictionary
for key, value in task_rel_error.items():
    # Convert the list to a NumPy array
    array_rel_error = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('relative_error_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_rel_error)

In [33]:
def findMedianSortedArrays(nums1, nums2):
    # Ensure nums1 is the smaller array
    if len(nums1) > len(nums2):
        nums1, nums2 = nums2, nums1

    x, y = len(nums1), len(nums2)
    low, high = 0, x

    while low <= high:
        partitionX = (low + high) // 2
        partitionY = (x + y + 1) // 2 - partitionX

        maxX = float('-inf') if partitionX == 0 else nums1[partitionX - 1]
        minX = float('inf') if partitionX == x else nums1[partitionX]

        maxY = float('-inf') if partitionY == 0 else nums2[partitionY - 1]
        minY = float('inf') if partitionY == y else nums2[partitionY]

        if maxX <= minY and maxY <= minX:
            # We have found the correct partition
            if (x + y) % 2 == 0:
                # If the total number of elements is even
                return (max(maxX, maxY) + min(minX, minY)) / 2
            else:
                # If the total number of elements is odd
                return max(maxX, maxY)
        elif maxX > minY:
            # We need to move partitionX to the left
            high = partitionX - 1
        else:
            # We need to move partitionX to the right
            low = partitionX + 1

# Example usage:
nums1 = [1, 3]
nums2 = [2, 3, 4]
result = findMedianSortedArrays(nums1, nums2)
print("Median:", result)

Median: 3
